<a href="https://colab.research.google.com/github/LoreenaPeragine/BookRecProject/blob/main/BookReccomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a book reccomendation program for a master's project. 

Imports 

In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras import backend as K
from keras.utils.vis_utils import plot_model
import tensorflow.keras.constraints
from matplotlib import pyplot as plt 
import seaborn as sns

from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


A look at the datasets, which originated from [Kaggle](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?resource=download)

In [29]:
booksData = pd.read_csv('/content/gdrive/MyDrive/BookRecAppData/Books.csv', low_memory=False)
print(booksData.head())
print(booksData.shape)

         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

In [30]:
usersData = pd.read_csv('/content/gdrive/MyDrive/BookRecAppData/Users.csv', low_memory=False)
print(usersData.head())
print(usersData.shape)

   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN
(278858, 3)


In [31]:
ratingsData = pd.read_csv('/content/gdrive/MyDrive/BookRecAppData/Ratings.csv', low_memory=False)
print(ratingsData.head())
print(ratingsData.shape)

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6
(1149780, 3)
